## 데이터 불러오기

In [1]:
%reset
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Conv2D,MaxPooling2D
from tensorflow.keras.layers import Flatten, Dense,Dropout
from tensorflow.keras.optimizers import Adam

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras import backend as K


train_dir = './team_made'
# ImageDataGenerator를 생성해요!
datagen = ImageDataGenerator(rescale=1/255,
                             validation_split=0.2)


train_generator = datagen.flow_from_directory(
    train_dir,    # target directory
    target_size=(28,28),
    batch_size=20,
    class_mode='sparse',
    subset='training')

validation_generator = datagen.flow_from_directory(
    train_dir,    # target directory
    target_size=(28,28),
    batch_size=20,
    class_mode='sparse',
    subset='validation')

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
Found 8700 images belonging to 24 classes.
Found 2169 images belonging to 24 classes.


In [2]:
test_datagen = ImageDataGenerator(rescale=1/255)

## 모델링

In [3]:
model = Sequential()

from tensorflow.keras.models import load_model
class FixedDropout(tf.keras.layers.Dropout):
    def _get_noise_shape(self, inputs):
        if self.noise_shape is None:
            return self.noise_shape
        symbolic_shape = K.shape(inputs)
        noise_shape = [symbolic_shape[axis] if shape is None else shape for axis, shape in enumerate(self.noise_shape)]
        return tuple(noise_shape)

new_model = tf.keras.models.load_model('./fine_tuning_model.h5',custom_objects={'FixedDropout':FixedDropout})

## 인식캠

In [7]:
import cv2 as cv
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# RGB 값 가지고 오기 초기값 생성
hsv = 0
lower_blue1 = 0
upper_blue1 = 0
lower_blue2 = 0
upper_blue2 = 0
lower_blue3 = 0
upper_blue3 = 0 


def empty(x): # 트랙바 생성시 필요함
    pass

def gray(input_img):
    for y in range(input_img.shape[0]):
        for x in range(input_img.shape[1]):
            input_img[y,x] = int(np.mean(input_img[y,x]))
    gray_2d_pixel = input_img[:,:,0]
    return gray_2d_pixel

# 원하는 물체를 구분하기 위함
def make_mask_image(img_bgr):
    img_hsv = cv.cvtColor(img_bgr, cv.COLOR_BGR2HSV)
    low = (0, 30, 0)
    high = (15, 255, 255)
    img_mask = cv.inRange(img_hsv, low, high)
    return img_mask

white_color = (255, 255, 255)
classes = ['A','B','C','D','E','F','G','H','I','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y']

def mouse_callback(event, x, y, flags, param):
    global hsv, lower_blue1, upper_blue1, lower_blue2, upper_blue2, lower_blue3, upper_blue3, threshold

    # 마우스 왼쪽 버튼 누를시 위치에 있는 픽셀값을 읽어와서 HSV로 변환 (x, y 값으로 저장)
    if event == cv.EVENT_LBUTTONDOWN:
        print(img_color[y, x])
        color = img_color[y, x]

        one_pixel = np.uint8([[color]])
        hsv = cv.cvtColor(one_pixel, cv.COLOR_BGR2HSV)
        hsv = hsv[0][0]
        
        threshold = cv.getTrackbarPos('threshold', 'img_result')

        
        # HSV 색공간에서 마우스 클릭으로 얻은 픽셀값과 유사한 픽셀값의 범위를 정함
        if hsv[0] < 10:
            print("case1")
            lower_blue1 = np.array([hsv[0]-10+180, threshold, threshold]) # 색상만 조절
            upper_blue1 = np.array([180, 255, 255])
            lower_blue2 = np.array([0, threshold, threshold])
            upper_blue2 = np.array([hsv[0], 255, 255])
            lower_blue3 = np.array([hsv[0], threshold, threshold])
            upper_blue3 = np.array([hsv[0]+10, 255, 255])
            #     print(i-10+180, 180, 0, i)
            #     print(i, i+10)

        elif hsv[0] > 170:
            print("case2")
            lower_blue1 = np.array([hsv[0], threshold, threshold])
            upper_blue1 = np.array([180, 255, 255])
            lower_blue2 = np.array([0, threshold, threshold])
            upper_blue2 = np.array([hsv[0]+10-180, 255, 255])
            lower_blue3 = np.array([hsv[0]-10, threshold, threshold])
            upper_blue3 = np.array([hsv[0], 255, 255])
            #     print(i, 180, 0, i+10-180)
            #     print(i-10, i)
        else:
            print("case3")
            lower_blue1 = np.array([hsv[0], threshold, threshold])
            upper_blue1 = np.array([hsv[0]+10, 255, 255])
            lower_blue2 = np.array([hsv[0]-10, threshold, threshold])
            upper_blue2 = np.array([hsv[0], 255, 255])
            lower_blue3 = np.array([hsv[0]-10, threshold, threshold])
            upper_blue3 = np.array([hsv[0], 255, 255])
            #     print(i, i+10)
            #     print(i-10, i)

        print(hsv[0])
        print("@1", lower_blue1, "~", upper_blue1)
        print("@2", lower_blue2, "~", upper_blue2)
        print("@3", lower_blue3, "~", upper_blue3)


        
cv.namedWindow('img_color')
cv.setMouseCallback('img_color', mouse_callback)

cv.namedWindow('img_result')
cv.createTrackbar('threshold', 'img_result', 0, 255, empty)
cv.setTrackbarPos('threshold', 'img_result', 30) # 초기값 30 / 트랙바


# 마스크 흑백
# 컬러 원본
# 리절트 값
cap = cv.VideoCapture(0)


while(True):
    
    ret, img_color = cap.read()
    height, width = img_color.shape[:2]
    img_color = cv.flip(img_color, 1)
    
    
#     print(img_color.shape[:2])  #(480, 640)
   
    img_color = cv.resize(img_color, (width, height), interpolation=cv.INTER_AREA)
    img_hsv = cv.cvtColor(img_color, cv.COLOR_BGR2HSV)

    img_mask1 = cv.inRange(img_hsv, lower_blue1, upper_blue1)
    img_mask2 = cv.inRange(img_hsv, lower_blue2, upper_blue2)
    img_mask3 = cv.inRange(img_hsv, lower_blue3, upper_blue3)
    img_mask = img_mask1 | img_mask2 | img_mask3
    
    kernel = np.ones((11,11), np.uint8) 
    img_mask = cv.morphologyEx(img_mask, cv.MORPH_OPEN, kernel) # 점으로 보이는 노이즈를 제거
    img_mask = cv.morphologyEx(img_mask, cv.MORPH_CLOSE, kernel) # 검은 구멍 채우기

    img_result = cv.bitwise_and(img_color, img_color, mask=img_mask)
    
    numOfLabels, img_label, stats, centroids = cv.connectedComponentsWithStats(img_mask)
    
#     img_roi=img_input[150:height-150, 250:width-250,:]

 # 사진을 계속 연결해서 동영상처럼 보이게 함
    for index, centroid in enumerate(centroids):
        if stats[index][0] == 0 and stats[index][1] == 0:
            continue
        
        if np.any(np.isnan(centroid)):
            continue
        
        x, y, width, height, area = stats[index]
        centerX, centerY = int(centroid[0]), int(centroid[1])
        
        # 영역을 잡아줌
        if area > 10000:
            cv.circle(img_color, (centerX, centerY), 10, (0, 0, 255), 10)
            cv.rectangle(img_color, (x, y), (x+width, y+height), (0,0,255))
            img_input = img_color.copy()
            img_roi=img_input[:,:,:]
            img_data = cv.resize(img_roi,(150,150))
            img_data = img_data.reshape(-1,150,150,3)
#             img_gray = gray(img_data)
#             img_gray = img_gray.reshape(1,784)
            img_norm = test_datagen.flow(img_data)
#             norm_img_data = scaler.transform(img_gray)
            prediction = new_model.predict_generator(img_norm)

            # 왼쪽 상단에 예측값을 보여줌
            idx = np.argmax(prediction)
            cv2.putText(img_color, classes[idx], (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, white_color, 2, cv2.LINE_AA)

    
    cv.imshow('img_color', img_color)
    cv.imshow('img_mask', img_mask)
    cv.imshow('img_result', img_result)


    if cv.waitKey(1) & 0xFF == 27:
        break
cv.destroyAllWindows()

[ 91  95 127]
case1
3
@1 [173  30  30] ~ [180 255 255]
@2 [ 0 30 30] ~ [  3 255 255]
@3 [ 3 30 30] ~ [ 13 255 255]
